In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:37267")
client

<Client: 'tcp://127.0.0.1:37267' processes=8 threads=40, memory=157.36 GiB>

In [2]:
client.restart()

<Client: 'tcp://127.0.0.1:35679' processes=0 threads=0, memory=0 B>

In [3]:
import numpy as np
import xarray as xr
import gsw
import time
import gcm_filters
from xmitgcm import open_mdsdataset
from xgcm.grid import Grid
# from MITgcmutils import jmd95
from dask.diagnostics import ProgressBar
import os.path as op
from scipy.interpolate import griddata
from scipy.stats import pearsonr
# import dask_image.ndfilters as dsfilt
import scipy.sparse as spar
import dask.array as dsar
# from xhistogram.xarray import histogram as xhist
# import xrft
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from matplotlib.lines import Line2D
%matplotlib inline

In [4]:
rdir = '/tank/chaocean/'
ddir = '/tank/topog/tuchida/TWA/'
spec = '/tank/spectre/tuchida/TWA/'

In [5]:
year = 1967

grav = 9.81
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
Cp = 3.994e3
ySstart = -15
yNstart = 15
yend = 50
ts, bs = (13,48)
nremap = 50
nsig = 25

mchunk = 1
xchunk = 250
ychunk = 400
schunk = -1

In [5]:
# ny, nx, nb = (3,3,1)
# def _filt(da, nker=np.ones((ny,nx,nb))):
#     return xr.DataArray(dsfilt.median_filter(da.data, footprint=nker), 
#                         dims=da.dims, coords=da.coords)

def _filt(da, area=None, dim=['YC','XC'], nker=15, cutoff=11):
    
    if area is not None:
        win_2D = (da * area).window
    else:
        win_2D = da.window
    
    win_2D.set(window='hanning', cutoff=cutoff, dim=dim, n=[nker,nker])
    bw = win_2D.boundary_weights(drop_dims=[])
    da_LS = win_2D.convolve(weights=bw)
    
    if area is not None:
        area_win = area.window
        area_win.set(window='hanning', cutoff=cutoff, dim=dim, n=[nker,nker])
        bw = area_win.boundary_weights(drop_dims=[])
        da_LS *= area_win.convolve(weights=bw)**-1
    
    return da_LS

In [6]:
dnf = 2160
nfile = 73
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1421280,1576800+dnf,dnf).reshape(len(years),nfile)
dd = 0   # 1490400
yystart = 0
ntimes[yystart,dd:]

array([1421280, 1423440, 1425600, 1427760, 1429920, 1432080, 1434240,
       1436400, 1438560, 1440720, 1442880, 1445040, 1447200, 1449360,
       1451520, 1453680, 1455840, 1458000, 1460160, 1462320, 1464480,
       1466640, 1468800, 1470960, 1473120, 1475280, 1477440, 1479600,
       1481760, 1483920, 1486080, 1488240, 1490400, 1492560, 1494720,
       1496880, 1499040, 1501200, 1503360, 1505520, 1507680, 1509840,
       1512000, 1514160, 1516320, 1518480, 1520640, 1522800, 1524960,
       1527120, 1529280, 1531440, 1533600, 1535760, 1537920, 1540080,
       1542240, 1544400, 1546560, 1548720, 1550880, 1553040, 1555200,
       1557360, 1559520, 1561680, 1563840, 1566000, 1568160, 1570320,
       1572480, 1574640, 1576800])

# Spatial correlation
$$ r = \frac{\sum[(F^\text{true} - \langle F^\text{true}\rangle) (F^\text{param} - \langle F^\text{param}\rangle)]}{\sqrt{\sum[(F^\text{true} - \langle F^\text{true}\rangle)^2]}\sqrt{\sum[(F^\text{param} - \langle F^\text{param}\rangle)^2]}} $$

In [7]:
yy = yystart
# xlim = [270,340]

for year in years:
    
    if year == years[yystart]:
        mtimes = ntimes[yy,:1]
        ystart = ySstart
    else:
        mtimes = ntimes[yy]
        ystart = ySstart
        
    for itime in mtimes:
        
        K = xr.open_zarr(op.join(spec,
                                 '5Dave/run%4d/Diff-tensor/explicit/no-skipNaN/Gauss-%2dkm_nC%02d_%02dN-%02dN/%010d.zarr' 
                                 % (year,50,10,ySstart,yend,itime))
                        ).K
        ds = open_mdsdataset(op.join(rdir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                                     % (0,year)), 
                             grid_dir=op.join(rdir,'grid_chaO/gridMIT_update1/'),
                             iters=itime,
                             prefix=['diag_ocnTave'], delta_t=2e2,
                            ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                  YG=slice(ystart-12**-1,yend+2*12**-1)
                                 ).chunk({'XC':xchunk,'XG':xchunk,
                                          'YC':ychunk,'YG':ychunk})
        nmembs = np.arange(36,60)
        for memb in nmembs:
            if memb == 36:
                dsz = xr.open_zarr(op.join(spec,'5Dave/Zs/run%4d/memb%02d/%010d/' 
                                                   % (year,memb,itime))
                                          ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                                YG=slice(ystart-12**-1,yend+2*12**-1)
                                               ).reset_coords(drop=True).isel(time=0)

            else:
                dsz = xr.concat([dsz, xr.open_zarr(op.join(spec,'5Dave/Zs/run%4d/memb%02d/%010d/' 
                                                           % (year,memb,itime))
                                                  ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                                        YG=slice(ystart-12**-1,yend+2*12**-1)
                                                       ).reset_coords(drop=True).isel(time=0)
                                ], 'nmemb')
        dsz.coords['nmemb'] = nmembs
        dsmnb = xr.open_zarr(op.join(spec,'5Dave/run%4d/HorDynb_%2d-%2dmembs/%010d/' 
                                    % (year,36,59,itime))
                           ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                 YG=slice(ystart-12**-1,yend+2*12**-1)
                                ).reset_coords(drop=True)
        dsptb = xr.open_zarr(op.join(spec,'5Dave/run%4d/PTracerb/%010d/' 
                                    % (year,itime))
                           ).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                ).reset_coords(drop=True)
        dstb = xr.open_zarr(op.join(spec,'5Dave/run%4d/Tracerb/%2d-%2dmembs/%010d/' 
                                    % (year,36,59,itime))
                           ).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                ).reset_coords(drop=True)
        dszb = xr.open_zarr(op.join(spec,'5Dave/run%4d/Zetab_%2d-%2dmembs/%010d/' 
                                    % (year,36,59,itime))
                           ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                                 YG=slice(ystart-12**-1,yend+2*12**-1)
                                ).reset_coords(drop=True)
        dstb.coords['delta'] = dsz.delta
        dsptb.coords['delta'] = dsz.delta
        dsmnb.coords['delta'] = dsz.delta
        dszb.coords['delta'] = dsz.delta
        dsFp = xr.open_zarr(op.join(spec,'5Dave/run%4d/EddyPTrFlx/%010d/'    # Explicit diagnosis of the eddy flux
                                    % (year,itime))
                           ).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                ).reset_coords(drop=True)
        dsTp = xr.open_zarr(op.join(spec,'5Dave/run%4d/EddyTrFlx/explicit/%2d-%2dmembs/%010d/' 
                                    % (year,36,59,itime))
                           ).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                ).reset_coords(drop=True)
    
        if itime == mtimes[0]:
            grid = Grid(dsz, periodic=['X'], 
                        coords={'Z':{'center':'delta','outer':'delta_outer'},
                               'Y':{'center':'YC','left':'YG'},
                               'X':{'center':'XC','left':'XG'}}
                       )
        buoybp1 = -grav * dsz.delta_outer * rhoConst**-1
        sigma = dsz.dzetaF * -grid.diff(buoybp1,'Z',boundary='fill')**-1
        sigmay = dsz.dzetaFy * -grid.diff(buoybp1,'Z',boundary='fill')**-1
        sigmax = dsz.dzetaFx * -grid.diff(buoybp1,'Z',boundary='fill')**-1

        sigmab = (dsz.dzetaF.mean('nmemb',skipna=True) 
                  * -grid.diff(buoybp1,'Z',boundary='fill')**-1
                 )
        sigmaxb = (dsz.dzetaFx.mean('nmemb',skipna=True) 
                   * -grid.diff(buoybp1,'Z',boundary='fill')**-1
                  )
        sigmayb = (dsz.dzetaFy.mean('nmemb',skipna=True) 
                   * -grid.diff(buoybp1,'Z',boundary='fill')**-1
                  )
        
##########################
        mask = ~dszb.dzetab.where(dszb.dzetab!=0.).where(dszb.zetab!=0.
                                              ).where(dstb.that!=0.
                                                     ).where(dstb.shat!=0.
                                                            ).where(dsptb.P0hat!=0
                                                                   ).where(dsptb.P1hat!=0
                                                                          ).where(dsptb.P2hat!=0
                                                                                 ).where(dsptb.P3hat!=0
                                                                                        ).where(dsTp.upTphat!=0.
                                                                                               ).where(dsTp.upSphat!=0.
                                                                                                      ).where(dsTp.vpTphat!=0.
                                                                                                             ).where(dsTp.vpSphat!=0.
                                                                                                                    ).to_masked_array().mask
        maskx = ~dszb.dzetaxb.where(dszb.dzetaxb!=0.).where(dszb.zetaxb!=0.).to_masked_array().mask
        masky = ~dszb.dzetayb.where(dszb.dzetayb!=0.).where(dszb.zetayb!=0.).to_masked_array().mask
        wet_mask = xr.DataArray(np.zeros((len(dsz.YC),len(dsz.XC),len(dsz.delta_outer))),
                                dims=['YC','XC','delta_outer'],
                                coords={'YC':dsz.YC,'XC':dsz.XC,
                                        'delta_outer':dsz.delta_outer}
        )
        wet_mask[1:-1] = mask[1:-1].astype(np.float64)
        wet_mask = wet_mask.chunk({'delta_outer':10})
        area = (grid.interp(ds.dxG,'Y',boundary='extend') 
                * grid.interp(ds.dyG,'X',boundary='extend')
               )
        dxw = xr.DataArray(ds.dxC.data,
                           dims=area.dims, coords=area.coords
                          ) # x-spacing centered at western cell edge

        dyw = xr.DataArray(ds.dyG.data,
                           dims=area.dims, coords=area.coords
                          ) # y-spacing centered at western cell edge

        dxs = xr.DataArray(ds.dxG.data,
                           dims=area.dims, coords=area.coords
                          ) # x-spacing centered at southern cell edge

        dys = xr.DataArray(ds.dyC.data,
                           dims=area.dims, coords=area.coords
                          ) # y-spacing centered at southern cell edge
        dx_min = min(dxw.min(), dyw.min(), dxs.min(), dys.min())
        dx_min = dx_min.values
        kappa_w = xr.ones_like(wet_mask)
        kappa_s = xr.ones_like(wet_mask)
        
        filter_scale = 50e3    # 50km
        filt_mask = wet_mask.chunk({'YC':-1,'XC':-1}).reset_coords(drop=True)

        gauss_filter = gcm_filters.Filter(
                            filter_scale=filter_scale,
                            dx_min=dx_min,
                        #     n_steps=36,
                            filter_shape=gcm_filters.FilterShape.GAUSSIAN,
                            grid_type=gcm_filters.GridType.IRREGULAR_WITH_LAND,
                            grid_vars={
                                'wet_mask': filt_mask, 
                                'dxw': dxw.chunk({'YC':-1,'XC':-1}), 'dyw': dyw.chunk({'YC':-1,'XC':-1}), 
                                'dxs': dxs.chunk({'YC':-1,'XC':-1}), 'dys': dys.chunk({'YC':-1,'XC':-1}), 
                                'area': area.chunk({'YC':-1,'XC':-1}), 
                                'kappa_w': kappa_w.chunk({'YC':-1,'XC':-1}), 
                                'kappa_s': kappa_s.chunk({'YC':-1,'XC':-1})
                            }
                        )
        
####
        P0b_sm = gauss_filter.apply(dsptb.P0hat.where(wet_mask!=0.).chunk({'YC':-1,'XC':-1}), dims=['YC','XC']
                                   )
        P1b_sm = gauss_filter.apply(dsptb.P1hat.where(wet_mask!=0.).chunk({'YC':-1,'XC':-1}), dims=['YC','XC']
                                   )
        P2b_sm = gauss_filter.apply(dsptb.P2hat.where(wet_mask!=0.).chunk({'YC':-1,'XC':-1}), dims=['YC','XC']
                                   )
        P3b_sm = gauss_filter.apply(dsptb.P3hat.where(wet_mask!=0.).chunk({'YC':-1,'XC':-1}), dims=['YC','XC']
                                   )
        P0bsm_x = (grid.diff(grid.interp(P0b_sm,'X').where(xr.DataArray(maskx, dims=['YC','XG','delta_outer'])!=0.),'X') 
                   * grid.interp(ds.dxG,'Y',boundary='extend')**-1)
        P0bsm_y = (grid.diff(grid.interp(P0b_sm,'Y',boundary='extend').where(xr.DataArray(masky, dims=['YG','XC','delta_outer'])!=0.),
                             'Y',boundary='extend') 
                   * grid.interp(ds.dyG,'X')**-1)
        P1bsm_x = (grid.diff(grid.interp(P1b_sm,'X').where(xr.DataArray(maskx, dims=['YC','XG','delta_outer'])!=0.),'X') 
                   * grid.interp(ds.dxG,'Y',boundary='extend')**-1)
        P1bsm_y = (grid.diff(grid.interp(P1b_sm,'Y',boundary='extend').where(xr.DataArray(masky, dims=['YG','XC','delta_outer'])!=0.),
                             'Y',boundary='extend') 
                   * grid.interp(ds.dyG,'X')**-1)
        P2bsm_x = (grid.diff(grid.interp(P2b_sm,'X').where(xr.DataArray(maskx, dims=['YC','XG','delta_outer'])!=0.),'X') 
                   * grid.interp(ds.dxG,'Y',boundary='extend')**-1)
        P2bsm_y = (grid.diff(grid.interp(P2b_sm,'Y',boundary='extend').where(xr.DataArray(masky, dims=['YG','XC','delta_outer'])!=0.),
                             'Y',boundary='extend') 
                   * grid.interp(ds.dyG,'X')**-1)
        P3bsm_x = (grid.diff(grid.interp(P3b_sm,'X').where(xr.DataArray(maskx, dims=['YC','XG','delta_outer'])!=0.),'X') 
                   * grid.interp(ds.dxG,'Y',boundary='extend')**-1)
        P3bsm_y = (grid.diff(grid.interp(P3b_sm,'Y',boundary='extend').where(xr.DataArray(masky, dims=['YG','XC','delta_outer'])!=0.),
                             'Y',boundary='extend') 
                   * grid.interp(ds.dyG,'X')**-1)
        
        that_sm = gauss_filter.apply(dstb.that.where(wet_mask!=0.).chunk({'YC':-1,'XC':-1}), dims=['YC','XC']
                                    )
        shat_sm = gauss_filter.apply(dstb.shat.where(wet_mask!=0.).chunk({'YC':-1,'XC':-1}), dims=['YC','XC']
                                    )
        thatsm_x = (grid.diff(grid.interp(that_sm,'X').where(xr.DataArray(maskx, dims=['YC','XG','delta_outer'])!=0.),'X') 
                    * grid.interp(ds.dxG,'Y',boundary='extend')**-1)
        thatsm_y = (grid.diff(grid.interp(that_sm,'Y',boundary='extend').where(xr.DataArray(masky, dims=['YG','XC','delta_outer'])!=0.),
                              'Y',boundary='extend') 
                    * grid.interp(ds.dyG,'X',boundary='extend')**-1)
        shatsm_x = (grid.diff(grid.interp(shat_sm,'X').where(xr.DataArray(maskx, dims=['YC','XG','delta_outer'])!=0.),'X') 
                    * grid.interp(ds.dxG,'Y',boundary='extend')**-1)
        shatsm_y = (grid.diff(grid.interp(shat_sm,'Y',boundary='extend').where(xr.DataArray(masky, dims=['YG','XC','delta_outer'])!=0.),
                              'Y',boundary='extend') 
                    * grid.interp(ds.dyG,'X',boundary='extend')**-1)
        
##########################
        upP0hat_sm = gauss_filter.apply((dsFp.upP0phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dyG,'X')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        vpP0hat_sm = gauss_filter.apply((dsFp.vpP0phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dxG,'Y',boundary='extend')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        upP1hat_sm = gauss_filter.apply((dsFp.upP1phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dyG,'X')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        vpP1hat_sm = gauss_filter.apply((dsFp.vpP1phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dxG,'Y',boundary='extend')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        upP2hat_sm = gauss_filter.apply((dsFp.upP2phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dyG,'X')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        vpP2hat_sm = gauss_filter.apply((dsFp.vpP2phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dxG,'Y',boundary='extend')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        upP3hat_sm = gauss_filter.apply((dsFp.upP3phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dyG,'X')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        vpP3hat_sm = gauss_filter.apply((dsFp.vpP3phat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dxG,'Y',boundary='extend')
                                        ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        
        upThat_sm = gauss_filter.apply((dsTp.upTphat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dyG,'X')
                                       ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        vpThat_sm = gauss_filter.apply((dsTp.vpTphat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dxG,'Y',boundary='extend')
                                       ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        upShat_sm = gauss_filter.apply((dsTp.upSphat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dyG,'X')
                                       ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])
        vpShat_sm = gauss_filter.apply((dsTp.vpSphat.where(wet_mask!=0.)
                                         # /grid.interp(ds.dxG,'Y',boundary='extend')
                                       ).chunk({'YC':-1,'XC':-1}), dims=['YC','XC'])

##########################
        nC = 10
        rAc = (ds.rA.where(wet_mask!=0.)
               .isel(YC=slice(1,-1),delta_outer=slice(ts,bs))
               .coarsen({'YC':nC,'XC':nC}, boundary='trim')
              ).sum(skipna=False) 
        zetabC = (((dszb.zetab * ds.rA).where(wet_mask!=0.)
                  ).isel(YC=slice(1,-1),delta_outer=slice(ts,bs)).coarsen({'YC':nC,'XC':nC},boundary='trim').sum() 
                  / rAc)
        # maskC100 = ~zetabC.isel(delta_outer=23).where(zetabC.isel(delta_outer=23)<-100.).to_masked_array().mask
        # maskC150 = ~zetabC.isel(delta_outer=23).where(zetabC.isel(delta_outer=23)<-150.).to_masked_array().mask
        
        uP0 = ((upP0hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False)
               / rAc).where(zetabC<-150.)
        vP0 = ((vpP0hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        uP1 = ((upP1hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        vP1 = ((vpP1hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        uP2 = ((upP2hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        vP2 = ((vpP2hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        uP3 = ((upP3hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        vP3 = ((vpP3hat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
               .isel(YC=slice(1,-1))
               .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)

        P0x_sm = ((P0bsm_x * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P0y_sm = ((P0bsm_y * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P1x_sm = ((P1bsm_x * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P1y_sm = ((P1bsm_y * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P2x_sm = ((P2bsm_x * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P2y_sm = ((P2bsm_y * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P3x_sm = ((P3bsm_x * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        P3y_sm = ((P3bsm_y * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
                  .isel(YC=slice(1,-1))
                  .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        
        uT = ((upThat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
              .isel(YC=slice(1,-1))
              .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        vT = ((vpThat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
              .isel(YC=slice(1,-1))
              .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        uS = ((upShat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
              .isel(YC=slice(1,-1))
              .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)
        vS = ((vpShat_sm * ds.rA.where(wet_mask!=0.)
               ).isel(delta_outer=slice(ts,bs))
              .isel(YC=slice(1,-1))
              .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
               / rAc).where(zetabC<-150.)

        Tx_sm = ((thatsm_x * ds.rA.where(wet_mask!=0.)
                 ).isel(delta_outer=slice(ts,bs))
                 .isel(YC=slice(1,-1))
                 .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        Ty_sm = ((thatsm_y * ds.rA.where(wet_mask!=0.)
                 ).isel(delta_outer=slice(ts,bs))
                 .isel(YC=slice(1,-1))
                 .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        Sx_sm = ((shatsm_x * ds.rA.where(wet_mask!=0.)
                 ).isel(delta_outer=slice(ts,bs))
                 .isel(YC=slice(1,-1))
                 .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        Sy_sm = ((shatsm_y * ds.rA.where(wet_mask!=0.)
                 ).isel(delta_outer=slice(ts,bs))
                 .isel(YC=slice(1,-1))
                 .coarsen({'YC':nC,'XC':nC},boundary='trim').sum(skipna=False) 
                  / rAc).where(zetabC<-150.)
        
        Fx0 = -(K[...,0,0] * P0x_sm
                + K[...,1,0] * P0y_sm
               )
        Fy0 = -(K[...,0,1] * P0x_sm
                + K[...,1,1] * P0y_sm
               )
        Fx1 = -(K[...,0,0] * P1x_sm
                + K[...,1,0] * P1y_sm
               )
        Fy1 = -(K[...,0,1] * P1x_sm
                + K[...,1,1] * P1y_sm
               )
        Fx2 = -(K[...,0,0] * P2x_sm
                + K[...,1,0] * P2y_sm
               )
        Fy2 = -(K[...,0,1] * P2x_sm
                + K[...,1,1] * P2y_sm
               )
        Fx3 = -(K[...,0,0] * P3x_sm
                + K[...,1,0] * P3y_sm
               )
        Fy3 = -(K[...,0,1] * P3x_sm
                + K[...,1,1] * P3y_sm
               )
        FxT = -(K[...,0,0] * Tx_sm
                + K[...,1,0] * Ty_sm
               )
        FyT = -(K[...,0,1] * Tx_sm
                + K[...,1,1] * Ty_sm
               )
        FxS = -(K[...,0,0] * Sx_sm
                + K[...,1,0] * Sy_sm
               )
        FyS = -(K[...,0,1] * Sx_sm
                + K[...,1,1] * Sy_sm
               )
        
##########################
#         start = time.time()
#         ename = op.join(spec,'5Dave/run%4d/Error/isig%2d-%2d_%010d.zarr' 
#                         % (year,ts,bs,itime))
#         ex0 = ((uP0 - Fx0) * np.abs(uP0)**-1)
#         ey0 = ((vP0 - Fy0) * np.abs(vP0)**-1)
#         ex1 = ((uP1 - Fx1) * np.abs(uP1)**-1)
#         ey1 = ((vP1 - Fy1) * np.abs(vP1)**-1)
#         ex2 = ((uP2 - Fx2) * np.abs(uP2)**-1)
#         ey2 = ((vP2 - Fy2) * np.abs(vP2)**-1)
#         ex3 = ((uP3 - Fx3) * np.abs(uP3)**-1)
#         ey3 = ((vP3 - Fy3) * np.abs(vP3)**-1)
#         exT = ((uT - FxT) * np.abs(uT)**-1)
#         eyT = ((vT - FyT) * np.abs(vT)**-1)
#         exS = ((uS - FxS) * np.abs(uS)**-1)
#         eyS = ((vS - FyS) * np.abs(vS)**-1)
        
#         fig, axes = plt.subplots(figsize=(13,10), nrows=2, ncols=2)
#         fig.set_tight_layout(True)
#         uP0.isel(delta_outer=10).plot(ax=axes[0,0], vmax=3e-3)
#         Fx0.isel(delta_outer=10).plot(ax=axes[0,1], vmax=3e-3)
#         ex0.isel(delta_outer=10).plot(ax=axes[1,0], vmax=1.)
#         ey0.isel(delta_outer=10).plot(ax=axes[1,1], vmax=1.)
#         plt.show()

#         dsave = ex0.to_dataset(name='ex0')
#         dsave['ey0'] = ey0
#         dsave['ex1'] = ex1
#         dsave['ey1'] = ey1
#         dsave['ex2'] = ex2
#         dsave['ey2'] = ey2
#         dsave['ex3'] = ex3
#         dsave['ey3'] = ey3
#         dsave['exT'] = exT
#         dsave['eyT'] = eyT
#         dsave['exS'] = exS
#         dsave['eyS'] = eyS
#         dsave.chunk({'YC':-1,'XC':-1,'delta_outer':1}).to_zarr(ename, mode='w')
#         if itime == mtimes[0]:
#             print(dsave)
#         dsave.close()
#         del ex0,ey0,ex1,ey1,ex2,ey2,ex3,ey3,exT,eyT,exS,eyS

##########################
        uP0_ = uP0.mean(['YC','XC'],skipna=True)
        Fx0_ = Fx0.mean(['YC','XC'],skipna=True)
        rx01 = (uP0 - uP0_)
        rx02 = (Fx0 - Fx0_)

        vP0_ = vP0.mean(['YC','XC'],skipna=True)
        Fy0_ = Fy0.mean(['YC','XC'],skipna=True)
        ry01 = (vP0 - vP0_)
        ry02 = (Fy0 - Fy0_)
####        
        uP1_ = uP1.mean(['YC','XC'],skipna=True)
        Fx1_ = Fx1.mean(['YC','XC'],skipna=True)
        rx11 = (uP1 - uP1_)
        rx12 = (Fx1 - Fx1_)

        vP1_ = vP1.mean(['YC','XC'],skipna=True)
        Fy1_ = Fy1.mean(['YC','XC'],skipna=True)
        ry11 = (vP1 - vP1_)
        ry12 = (Fy1 - Fy1_)
####        
        uP2_ = uP2.mean(['YC','XC'],skipna=True)
        Fx2_ = Fx2.mean(['YC','XC'],skipna=True)
        rx21 = (uP2 - uP2_)
        rx22 = (Fx2 - Fx2_)

        vP2_ = vP2.mean(['YC','XC'],skipna=True)
        Fy2_ = Fy2.mean(['YC','XC'],skipna=True)
        ry21 = (vP2 - vP2_)
        ry22 = (Fy2 - Fy2_)
####        
        uP3_ = uP3.mean(['YC','XC'],skipna=True)
        Fx3_ = Fx3.mean(['YC','XC'],skipna=True)
        rx31 = (uP3 - uP3_)
        rx32 = (Fx3 - Fx3_)

        vP3_ = vP3.mean(['YC','XC'],skipna=True)
        Fy3_ = Fy3.mean(['YC','XC'],skipna=True)
        ry31 = (vP3 - vP3_)
        ry32 = (Fy3 - Fy3_)
####    
        uT_ = uT.mean(['YC','XC'],skipna=True)
        FxT_ = FxT.mean(['YC','XC'],skipna=True)
        rx4 = (uT - uT_)
        rx5 = (FxT - FxT_)

        vT_ = vT.mean(['YC','XC'],skipna=True)
        FyT_ = FyT.mean(['YC','XC'],skipna=True)
        ry4 = (vT - vT_) 
        ry5 = (FyT - FyT_)
####    
        uS_ = uS.mean(['YC','XC'],skipna=True)
        FxS_ = FxS.mean(['YC','XC'],skipna=True)
        rx7 = (uS - uS_) 
        rx8 = (FxS - FxS_)

        vS_ = vS.mean(['YC','XC'],skipna=True)
        FyS_ = FyS.mean(['YC','XC'],skipna=True)
        ry7 = (vS - vS_)
        ry8 = (FyS - FyS_)
    
# ##########################
        # tmp1 = (rx1 * (rx2*rx3)**-1).compute()
        # tmp2 = (ry1 * (ry2*ry3)**-1).compute()
        # tmp3 = (rx4 * (rx5*rx6)**-1).compute()
        # tmp4 = (ry4 * (ry5*ry6)**-1).compute()
        # tmp5 = (rx7 * (rx8*rx9)**-1).compute()
        # tmp6 = (ry7 * (ry8*ry9)**-1).compute()

#         if itime == mtimes[0]:
#             rxP = tmp1
#             ryP = tmp2
#             rxT = tmp3
#             ryT = tmp4
#             rxS = tmp5
#             ryS = tmp6
#         else:
#             rxP = xr.concat([rxP, tmp1], 'time')
#             ryP = xr.concat([ryP, tmp2], 'time')
#             rxT = xr.concat([rxT, tmp3], 'time')
#             ryT = xr.concat([ryT, tmp4], 'time')
#             rxS = xr.concat([rxS, tmp5], 'time')
#             ryS = xr.concat([ryS, tmp6], 'time')

        if itime == mtimes[0] or itime % (20*dnf) == 0:
            print(r"$r^x_0$:", ((rx01*rx02).sum(['YC','XC'],skipna=True)
                     / (np.sqrt((rx01**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((rx02**2).sum(['YC','XC'],skipna=True)))
                    ).isel(delta_outer=10).values, 
                          r"$r^y_0$:", ((ry01*ry02).sum(['YC','XC'],skipna=True)
                     / (np.sqrt((ry01**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((ry02**2).sum(['YC','XC'],skipna=True)))
                    ).isel(delta_outer=10).values
                 )
            print(r"$r^x_\theta$:", ((rx4*rx5).sum(['YC','XC'],skipna=True)
                     / (np.sqrt((rx4**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((rx5**2).sum(['YC','XC'],skipna=True)))
                    ).isel(delta_outer=10).values, 
                          r"$r^y_\theta$:", ((ry4*ry5).sum(['YC','XC'],skipna=True)
                     / (np.sqrt((ry4**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((ry5**2).sum(['YC','XC'],skipna=True)))
                    ).isel(delta_outer=10).values
                 )
            print(r"$r^x_S$:",      ((rx7*rx8).sum(['YC','XC'],skipna=True)
                     / (np.sqrt((rx7**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((rx8**2).sum(['YC','XC'],skipna=True)))
                    ).isel(delta_outer=10).values, 
                          r"$r^y_S$:",      ((ry7*ry8).sum(['YC','XC'],skipna=True)
                     / (np.sqrt((ry7**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((ry8**2).sum(['YC','XC'],skipna=True)))
                    ).isel(delta_outer=10).values
                 )
    
        # end = time.time()
        # print("Lapse time:", end-start)
        # print(itime)
    
##########################
        dsave = rx01.to_dataset(name='rx01')
        dsave['rx02'] = rx02
        dsave['ry01'] = ry01
        dsave['ry02'] = ry02
        dsave['rx11'] = rx11
        dsave['rx12'] = rx12
        dsave['ry11'] = ry11
        dsave['ry12'] = ry12
        dsave['rx21'] = rx21
        dsave['rx22'] = rx22
        dsave['ry21'] = ry21
        dsave['ry22'] = ry22
        dsave['rx31'] = rx31
        dsave['rx32'] = rx32
        dsave['ry31'] = ry31
        dsave['ry32'] = ry32
        dsave['rxT1'] = rx4
        dsave['rxT2'] = rx5
        dsave['ryT1'] = ry4
        dsave['ryT2'] = ry5
        dsave['rxS1'] = rx7
        dsave['rxS2'] = rx8
        dsave['ryS1'] = ry7
        dsave['ryS2'] = ry8
        dsave.chunk({'YC':-1,'XC':-1,'delta_outer':1}).to_zarr(op.join(spec,'5Dave/run%4d/CorrCoef/isig%2d-%2d_%010d.zarr' 
                                                                       % (year,ts,bs,itime)),
                                                               mode='w')
        dsave.close()
        
        ds.close()
        dsz.close()
        dstb.close()
        dsptb.close()
        dsmnb.close()
        dszb.close()
        dsFp.close()
        dsTp.close()
        # del tmp1,tmp2,tmp3,tmp4,tmp5,tmp6
# #     dsave = rxP.to_dataset(name='rxP')
# #     dsave['ryP'] = ryP
# #     dsave['rxT'] = rxT
# #     dsave['ryT'] = ryT
# #     dsave['rxS'] = rxS
# #     dsave['ryS'] = ryS
# #     dsave.to_netcdf(op.join(ddir,'5Dave/run%4d/CorrCoef_isig%2d-%2d_%010d.nc' 
# #                             % (year,sstart,send,itime)))
# #     dsave.close()
        print(itime)
        client.restart()
    
    yy += 1

/tmp/ipykernel_51144/1347122753.py:31: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  dsz = xr.open_zarr(op.join(spec,'5Dave/Zs/run%4d/memb%02d/%010d/'
/tmp/ipykernel_51144/1347122753.py:38: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidate

$r^x_0$: 0.9183491292238446 $r^y_0$: 0.9171624537202976
$r^x_\theta$: 0.3439133858338677 $r^y_\theta$: 0.4939515010013707
$r^x_S$: 0.18058086220249184 $r^y_S$: 0.30811376035408833
1421280


In [8]:
dsr = xr.open_zarr(op.join(spec,'5Dave/run%4d/CorrCoef/isig%2d-%2d_%010d.zarr' 
                           % (year,ts,bs,itime))
                  )

print(r"$r^x_C0$:", ((dsr.rx01*dsr.rx02).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx01.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx02.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C0$:", ((dsr.ry01*dsr.ry02).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry01.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry02.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_C1$:", ((dsr.rx11*dsr.rx12).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx11.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx12.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C1$:", ((dsr.ry11*dsr.ry12).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry11.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry12.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_C2$:", ((dsr.rx21*dsr.rx22).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx21.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx22.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C2$:", ((dsr.ry21*dsr.ry22).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry21.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry22.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_C3$:", ((dsr.rx31*dsr.rx32).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx31.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx32.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C3$:", ((dsr.ry31*dsr.ry32).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry31.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry32.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)

print(r"$r^x_\theta$:", ((dsr.rxT1*dsr.rxT2).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rxT1.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rxT2.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_\theta$:", ((dsr.ryT1*dsr.ryT2).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ryT1.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ryT2.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_S$:", ((dsr.rxS1*dsr.rxS2).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rxS1.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rxS2.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_S$:", ((dsr.ryS1*dsr.ryS2).sel(YC=slice(10,30)).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ryS1.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ryS2.sel(YC=slice(10,30))**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)

$r^x_C0$: 0.9595970029003854 $r^y_C0$: 0.9713055012763941
$r^x_C1$: 0.9587676381039596 $r^y_C1$: 0.9363874600908724
$r^x_C2$: 0.9479907995876101 $r^y_C2$: 0.9694836027424573
$r^x_C3$: 0.966598441759203 $r^y_C3$: 0.9768196424042082
$r^x_\theta$: 0.303297518306416 $r^y_\theta$: 0.3534251440736545
$r^x_S$: 0.3902084916673922 $r^y_S$: 0.1282521497950843


In [9]:
dsr = xr.open_zarr(op.join(spec,'5Dave/run%4d/CorrCoef/isig%2d-%2d_%010d.zarr' 
                           % (year,ts,bs,itime))
                  )

print(r"$r^x_C0$:", ((dsr.rx01*dsr.rx02).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx01**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx02**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C0$:", ((dsr.ry01*dsr.ry02).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry01**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry02**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_C1$:", ((dsr.rx11*dsr.rx12).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx11**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx12**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C1$:", ((dsr.ry11*dsr.ry12).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry11**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry12**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_C2$:", ((dsr.rx21*dsr.rx22).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx21**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx22**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C2$:", ((dsr.ry21*dsr.ry22).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry21**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry22**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_C3$:", ((dsr.rx31*dsr.rx32).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rx31**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rx32**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_C3$:", ((dsr.ry31*dsr.ry32).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ry31**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ry32**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)

print(r"$r^x_\theta$:", ((dsr.rxT1*dsr.rxT2).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rxT1**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rxT2**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_\theta$:", ((dsr.ryT1*dsr.ryT2).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ryT1**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ryT2**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)
print(r"$r^x_S$:", ((dsr.rxS1*dsr.rxS2).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.rxS1**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.rxS2**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values, 
                  r"$r^y_S$:", ((dsr.ryS1*dsr.ryS2).sum(['YC','XC'],skipna=True)
             / (np.sqrt((dsr.ryS1**2).sum(['YC','XC'],skipna=True))
                        * np.sqrt((dsr.ryS2**2).sum(['YC','XC'],skipna=True)))
            ).isel(delta_outer=10).values)

$r^x_C0$: 0.9183490374463202 $r^y_C0$: 0.9171624537202977
$r^x_C1$: 0.935228706758737 $r^y_C1$: 0.9168934326063652
$r^x_C2$: 0.9752952502006013 $r^y_C2$: 0.9826317462386363
$r^x_C3$: 0.9243881778910792 $r^y_C3$: 0.9306036493867941
$r^x_\theta$: 0.3439133858338676 $r^y_\theta$: 0.49395150100137053
$r^x_S$: 0.1805808622024918 $r^y_S$: 0.3081137603540883
